In [2]:
!nvidia-smi

Fri Apr 22 16:25:16 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:1B:00.0 Off |                  N/A |
|  0%   24C    P8    24W / 420W |   6187MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:61:00.0 Off |                  N/A |
|  0%   

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

In [4]:
device = 'cuda:2' if torch.cuda.is_available() else 'cpu'

In [5]:
torch.cuda.set_device(device)

In [6]:
from utils import load_cifar10, load_cifar100,load_tinyimagenet

In [7]:
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

In [8]:
# load cifar10 data
trainloader, testloader = load_cifar10(data_dir="../data/cifar10", batch_size=128, 
                                       test_batch = 10000,train_shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


In [9]:
# load dataloader for steal
stealloader,_ = load_cifar100(data_dir="../data/cifar100", batch_size=128, train_shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [10]:
# load tinyimagenet for outlier exposure
outlier_loader = load_tinyimagenet(data_dir="../data/tiny-imagenet-200", 
                                   batch_size=128, train_shuffle=True)

/home/tako/anaconda3/envs/torch/lib/python3.7/site-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 100 worker processes in total. Our suggested max number of worker in current system is 80, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


# For MPC - use softmax (NOT logit)

In [11]:
from utils import sliced_wasserstein_distance

In [12]:
from cifar_models import Net, Net_logit

## Load Real Net

In [13]:
from cifar_models import Net, Net_logit, AttackNet

In [14]:
from utils import train_model_CE

In [15]:
import os
net_logit = AttackNet() 
net_logit.load_state_dict(torch.load("../../knock_attack/Maxpool_relu_baseline.pth"))

<All keys matched successfully>

CIFAR 100에 대해서는 logit값이 작은 것들이 있음, 즉, outlier에 대해 덜 confident하게 예측

# Load Fake Net

### fakenet: maximize loss

In [16]:
import os
fake_ML = AttackNet()
#fake_ML_swd.load_state_dict(torch.load(os.path.join("../results","cifar_fake_logit_swd.pth"),map_location='cpu'))
fake_ML.load_state_dict(torch.load(os.path.join("../results","cifar_fake_logit_ML.pth")))

<All keys matched successfully>

### fakenet: maximize loss + SWD

In [17]:
import os
fake_ML_swd = AttackNet()
#fake_ML_swd.load_state_dict(torch.load(os.path.join("../results","cifar_fake_logit_swd.pth"),map_location='cpu'))
fake_ML_swd.load_state_dict(torch.load(os.path.join("../results","cifar_fake_logit_swd_attacknet_save.pth")))

<All keys matched successfully>

### fakenet: random label

In [18]:
fake_random = AttackNet()
fake_random.load_state_dict(torch.load(os.path.join("../results","cifar_fake_random_net_logit_attacknet_save.pth")))

<All keys matched successfully>

### fakenet: random label + SWD

In [19]:
fake_random_swd = AttackNet()
fake_random_swd.load_state_dict(torch.load(os.path.join("../results","cifar_random_logit_swd_attacknet_save.pth")))

<All keys matched successfully>

# Adversarial attack for orig net

In [20]:
from adversarial_attacks_pytorch.torchattacks import PGD

In [21]:
atk_logit_orig = PGD(net_logit, eps=8/255, alpha=2/255, steps=7)

In [22]:
atk_logit_orig.save(data_loader=testloader, save_path="../results/orig_logit_pgd.pt", verbose=True) #net orig + test image

/home/tako/anaconda3/envs/torch/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448265233/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


- Save progress: 100.00 % / Robust accuracy: 0.57 % / L2: 19.17318 (23.626 it/s) 	


In [23]:
adv_images, adv_labels = torch.load("../results/orig_logit_pgd.pt")
adv_data = TensorDataset(adv_images.float()/255, adv_labels)
adv_loader = DataLoader(adv_data, batch_size=128, shuffle=False)

NameError: name 'TensorDataset' is not defined

In [ ]:
net_logit.cuda().eval()

correct = 0
total = 0

for images, labels in testloader:
    
    images = images.cuda()
    outputs = net_logit(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Standard accuracy: %.2f %%' % (100 * float(correct) / total))

In [ ]:
net_logit.cuda().eval()

correct = 0
total = 0

for images, labels in adv_loader:
    
    images = images.cuda()
    outputs = net_logit(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Robust accuracy: %.2f %%' % (100 * float(correct) / total))

# Adversarial attack for  comb net

## ml

In [ ]:
from utils import CombNet

In [ ]:
tau = 0.4

In [ ]:
comb_ml = CombNet(net_logit, fake_ML,  tau = tau).to(device)

In [ ]:
atk_comb_ml = PGD(comb_ml, eps=8/255, alpha=2/255, steps=7)

In [ ]:
atk_comb_ml.save(data_loader=testloader, save_path="../results/comb_logit_ml_pgd_tau%s.pt"%tau, verbose=True) #net orig + test image

In [ ]:
adv_images, adv_labels = torch.load("../results/comb_logit_ml_pgd_tau0.4.pt")
adv_data = TensorDataset(adv_images.float()/255, adv_labels)
adv_loader = DataLoader(adv_data, batch_size=128, shuffle=False)

In [ ]:
comb_ml.cuda().eval()

correct = 0
total = 0

for images, labels in testloader:
    
    images = images.cuda()
    outputs = comb_ml(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Standard accuracy: %.2f %%' % (100 * float(correct) / total))

In [ ]:
comb_ml.cuda().eval()

correct = 0
total = 0

for images, labels in adv_loader:
    
    images = images.cuda()
    outputs = comb_ml(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Robust accuracy: %.2f %%' % (100 * float(correct) / total))

In [ ]:
st_model_ml = AttackNet()
st_model_ml.load_state_dict(torch.load(os.path.join("../results", 'cifar_attacknet_logit_random_swd_new_tau%s.pth'%tau)))


In [ ]:
comb_ml.cuda().eval()

correct = 0
total = 0

for images, labels in adv_loader:
    
    images = images.cuda()
    outputs = comb_ml(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Robust accuracy: %.2f %%' % (100 * float(correct) / total))

## ml + swd

In [100]:
tau = 0.6

In [101]:
comb_ml_swd = CombNet(net_logit, fake_ML_swd,  tau = tau).to(device)

In [102]:
atk_comb_ml_swd = PGD(comb_ml_swd, eps=8/255, alpha=2/255, steps=7)

In [103]:
atk_comb_ml_swd.save(data_loader=testloader, save_path="../results/comb_logit_ml_swd_pgd_tau%s.pt"%tau, verbose=True) #net orig + test image

- Save progress: 100.00 % / Robust accuracy: 4.06 % / L2: 19.04495 (1.362 it/s) 	


In [104]:
adv_images, adv_labels = torch.load("../results/comb_logit_ml_swd_pgd_tau%s.pt"%tau)
adv_data = TensorDataset(adv_images.float()/255, adv_labels)
adv_loader = DataLoader(adv_data, batch_size=128, shuffle=False)

In [105]:
net = comb_ml_swd.cuda().eval()

correct = 0
total = 0

for images, labels in testloader:
    
    images = images.cuda()
    outputs = net(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Standard accuracy: %.2f %%' % (100 * float(correct) / total))

Standard accuracy: 82.32 %


In [106]:
#net = comb_ml_swd.cuda().eval()

correct = 0
total = 0

for images, labels in adv_loader:
    
    images = images.cuda()
    outputs = net(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Robust accuracy: %.2f %%' % (100 * float(correct) / total))

Robust accuracy: 10.00 %


## random

In [115]:
tau = 0.4

In [116]:
comb_random = CombNet(net_logit, fake_random,  tau = tau).to(device)

In [117]:
atk_comb_random = PGD(comb_random, eps=8/255, alpha=2/255, steps=7)

In [118]:
atk_comb_random.save(data_loader=testloader, save_path="../results/comb_logit_random_pgd_tau%s.pt"%tau, verbose=True) #net orig + test image

- Save progress: 100.00 % / Robust accuracy: 0.65 % / L2: 19.16980 (1.366 it/s) 	


In [119]:
adv_images, adv_labels = torch.load("../results/comb_logit_random_pgd_tau%s.pt"%tau)
adv_data = TensorDataset(adv_images.float()/255, adv_labels)
adv_loader = DataLoader(adv_data, batch_size=128, shuffle=False)

In [120]:
net = comb_random.cuda().eval()

correct = 0
total = 0

for images, labels in testloader:
    
    images = images.cuda()
    outputs = net(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Standard accuracy: %.2f %%' % (100 * float(correct) / total))

Standard accuracy: 84.12 %


In [121]:
#net = comb_ml_swd.cuda().eval()

correct = 0
total = 0

for images, labels in adv_loader:
    
    images = images.cuda()
    outputs = net(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Robust accuracy: %.2f %%' % (100 * float(correct) / total))

Robust accuracy: 10.00 %


## random + swd

In [142]:
tau = 0.9

In [143]:
comb_random_swd = CombNet(net_logit, fake_random_swd,  tau = tau).to(device)

In [144]:
atk_comb_random_swd = PGD(comb_random_swd, eps=8/255, alpha=2/255, steps=7)

In [145]:
atk_comb_random_swd.save(data_loader=testloader, save_path="../results/comb_logit_random_swd_pgd_tau%s.pt"%tau, verbose=True) #net orig + test image

- Save progress: 100.00 % / Robust accuracy: 4.50 % / L2: 19.14928 (1.369 it/s) 	


In [146]:
adv_images, adv_labels = torch.load("../results/comb_logit_random_swd_pgd_tau%s.pt"%tau)
adv_data = TensorDataset(adv_images.float()/255, adv_labels)
adv_loader = DataLoader(adv_data, batch_size=128, shuffle=False)

In [147]:
net = comb_random_swd.cuda().eval()

correct = 0
total = 0

for images, labels in testloader:
    
    images = images.cuda()
    outputs = net(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Standard accuracy: %.2f %%' % (100 * float(correct) / total))

Standard accuracy: 75.52 %


In [148]:
correct = 0
total = 0

for images, labels in adv_loader:
    
    images = images.cuda()
    outputs = net(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Robust accuracy: %.2f %%' % (100 * float(correct) / total))

Robust accuracy: 10.00 %


In [149]:
predicted

tensor([7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7], device='cuda:2')